## Importing libraries ##

In [10]:
import pandas as pd
import numpy as np
import scipy.signal as signal
from scipy.optimize import curve_fit
from scipy.special import factorial
from landaupy import landau
from scipy.special import factorial
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import h5py
import array
from collections import Counter
import daw_readout
from matplotlib.colors import LogNorm
from tqdm import trange
from tqdm import tqdm
import time  
import os 

np.set_printoptions(suppress=True) # to make tiny values appear as zero
np.set_printoptions(precision=4)   # number of digits in floats (or longfloats)
#np.set_printoptions(formatter={'float_kind':"{:.2f}"})
#np.set_printoptions()

%matplotlib inline

# Set formats
DARK=False
if(DARK==False):
    form='.k'
    plt.style.use(['default']) 
    fontcolor='k'
else:
    form='.w'
    plt.style.use(['dark_background'])
    fontcolor='w'


SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 22

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

params = {
    'font.family': 'serif',
    'font.size' : 14, 'axes.titlesize' : 14, 'axes.labelsize' : 14, 'axes.linewidth' : 1,
    # ticks
    'xtick.labelsize' : 14, 'ytick.labelsize' : 14, 'xtick.major.size' : 8, 'xtick.minor.size' : 4,
    'ytick.major.size' : 8, 'ytick.minor.size' : 4, 'xtick.major.width' : 1, 'xtick.minor.width' : 1,
    'ytick.major.width' : 1, 'ytick.minor.width' : 1, 'xtick.direction' : 'in', 'ytick.direction' : 'in',
    # markers
    'lines.markersize' : 4, 'lines.markeredgewidth' : 1, 'errorbar.capsize' : 5, 'lines.linewidth' : 1.5,
    #'lines.linestyle' : None, 'lines.marker' : None,
    'savefig.bbox' : 'tight', 'legend.fontsize' : 14,
    'axes.labelsize': 14, 'axes.titlesize':16, 'xtick.labelsize':14, 'ytick.labelsize':14,
    'backend': 'Agg', 'mathtext.fontset': 'dejavuserif',
    'figure.facecolor':'w',
    #pad
    'axes.labelpad':8,
    # ticks
    'xtick.major.pad': 7,   'xtick.minor.pad': 7,   
    'ytick.major.pad': 4, 'ytick.minor.pad': 4,
}
plt.rcParams.update(params)

### Load, process, examine and filter data

In [2]:
Date = 20240920 # YYYYMMDD
V = 1.13 # V
offset = 900 # mV
T = 61 # ns
PDiv = 0 # dB

In [3]:
name = "LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.13V_61ns_400_ratio_run0"

#name = "lv2415_anode_lv2414_dual_20240904_LED_2.300V_900mv_10ns_400_ratio_run0"
#name = "lv2415_anode_lv2414_dual_20240904_LED_2.300V_900mv_10ns_attenuator_9DB_400_ratio_run0"

#name = "lv2415_anode_lv2414_dual_"+str(Date)+"_LED_"+str(V)+"V_"+str(T)+"ns_attenuator_"+str(PDiv)+"DB_400_ratio_run0" 

In [ ]:
calbl = 10 ### Length used to calculate baseline
calstart = 95 ### Index started to calculate area
callen= 85 ### Length used to calculate area

In [ ]:
path = "/mnt/data/PMT/R8520_406/"   ##### file path

filesnum = 1 ### number of files

hdf5_filename = path + name + ".h5"
feather_filename = path + name + ".feather"
filename = {}
filepath = {}
dataload = {}
for i in range(filesnum):
    filename[i] = name + "_raw_b0_seg" + str(i) +".bin"
    filepath[i] = path + filename[i]
    dataload[i] = daw_readout.DAWDemoWaveParser(filepath[i])

#name_3 = 'run_gamma_$(date_raw_b0_seg0.bin'

### Background data
#data0 = daw_readout.DAWDemoWaveParser([path+name[0]])#### background data

data = []

for num in range(filesnum):
# Process the waveforms and store the data in lists
    i = 0
    for wave in tqdm(dataload[num]):
        baseline2 = np.sum(wave.Waveform[:calbl]) / calbl
        area = (baseline2 * callen - np.sum(wave.Waveform[calstart : calstart+callen]))
        height = (baseline2 - np.min(wave.Waveform[calstart : calstart+callen]))
        data.append({
            'Files': num,   ####file name/number
            'EventID': i,   ####index in file
            'Channel': wave.Channel, #### PMT Channel
            'Timestamp': wave.Timestamp, 
            'Baseline': wave.Baseline,
            'Baseline2': baseline2,
            'Height': height,
            'Area': area,
            'AtH':area/height,
            'Waveform':wave.Waveform
            })
        i += 1

# Convert lists to pandas DataFrames
df = pd.DataFrame(data)

# Print the total number of events for each channel
print("CH0 Total Events:",len(df.EventID[df.Channel==0]))
print("CH1 Total Events:",len(df.EventID[df.Channel==1]))

ch1_area_0 = []
ch0_area_0 = []
ch2_area_0 = []

ch0_area = []
ch1_area = []
ch2_area = []

area_ratio = []

for i in tqdm(range(0,len(df.Area))):
    if df.Channel[i] == 1:
        ch1_area.append(df.Area[i]/96.99)
    elif df.Channel[i] == 0:
        ch0_area.append(df.Area[i]/77.79)
    elif df.Channel[i] == 2:
        ch2_area.append(-df.Area[i]/77.79)

"""
for i in tqdm(range(0,len(df.Area))):
    if df.Channel[i] == 1:
        ch1_area_0.append(df.Area[i])
    elif df.Channel[i] == 0:
        ch0_area_0.append(df.Area[i])

for i in tqdm(range(0,len(ch0_area_0))):
    if ch1_area_0[i] > 60:
        ch0_area.append(ch0_area_0[i])
        ch1_area.append(ch1_area_0[i])
"""

for i in tqdm(range(0,len(ch0_area))):
    if ch1_area[i] != 0:
        area_ratio.append(ch0_area[i]/ch1_area[i])
    else:
        area_ratio.append(0)

In [ ]:
# Dynode ratio
print(V,T,round(np.mean(ch0_area),2),round(np.std(ch0_area),2),round(np.mean(ch2_area),2),round(np.std(ch2_area),2),PDiv)

In [ ]:
# Area ratio
print(V,T,round(np.mean(ch0_area),2),round(np.std(ch0_area),2),round(np.mean(ch1_area),2),round(np.std(ch1_area),2),PDiv)

In [ ]:
df[:]

In [ ]:
plt.figure(figsize=(15,4))
plt.title('Waveforms')
plt.xlabel('$t$ [4 ns]')
plt.ylabel('Amplitude [adc]')
plt.xlim(90,170)
plt.ylim(0,16000)
for i in tqdm(range (0,200)):
    #plt.step(np.linspace(0,150,150),df.Baseline[i]-df.Waveform[i])
    plt.plot(df.Baseline[i]-df.Waveform[i])
plt.legend()
plt.show()

### Scatter plots

In [ ]:
bin1 = np.linspace(np.min(ch1_area), np.max(ch1_area),200) ###x Height
bin2 = np.linspace(np.min(ch0_area), np.max(ch0_area), 200)###y Area
bins = [bin1,bin2]

H, xedges, yedges = np.histogram2d(ch1_area,ch0_area,bins)
H = np.rot90(H) 
H = np.flipud(H)
Hmasked = np.ma.masked_where(H==0,H)

# definitions for the axes
left, width = 1, 1
bottom, height = 1, 1
bottom_h = left_h = left + width + 0.025

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# start with a rectangular Figure
fig = plt.figure(1, figsize=(8, 8))

axScatter = plt.axes(rect_scatter)
axHistx = plt.axes(rect_histx)
axHisty = plt.axes(rect_histy)

axScatter.pcolormesh(xedges,yedges,Hmasked,norm=LogNorm())
axScatter.set_xlim(min(ch1_area),max(ch1_area))
axScatter.set_ylim(min(ch0_area),max(ch0_area))
axScatter.set_xlabel('CH1 area [PE]')
axScatter.set_ylabel('CH0 area [PE]')

axHistx.hist(ch1_area,bins=200,histtype='step',linewidth=3,color='darkblue')
axHistx.set_xlim(min(ch1_area),max(ch1_area))
axHistx.set_xticks([])
axHistx.set_ylabel('Counts')

axHisty.hist(ch0_area,bins=200,orientation='horizontal',histtype='step',linewidth=3,color='darkblue')
axHisty.set_ylim(min(ch0_area),max(ch0_area))
axHisty.set_yticks([])
axHisty.set_xlabel('Counts')

plt.suptitle('Pulse area with and without polariser, '+str(V)+' V, '+str(T)+' ns',x=1.55,y=2.275)

plt.savefig('./img/Pulse area with and without polariser, '+str(V)+' V, '+str(T)+'ns.png',dpi=300)
plt.show()

In [ ]:
bin3 = np.linspace(0,200,200) ###x Height
bin4 = np.linspace(np.min(ch0_area), np.max(ch0_area), 200)###y Area
bin2s = [bin4,bin3]

filtered_area_ratio = []
filtered_ch0_area = []
for i in tqdm(range(0,len(area_ratio))):
    if area_ratio[i]!=0:
        filtered_area_ratio.append(area_ratio[i])
        filtered_ch0_area.append(ch0_area[i])

area_average = np.average(filtered_area_ratio)

H, xedges, yedges = np.histogram2d(filtered_ch0_area,filtered_area_ratio,bin2s)
H = np.rot90(H) 
H = np.flipud(H)
Hmasked = np.ma.masked_where(H==0,H)\

H_sum = []

for i in range(0,len(H)):
    H_sum.append(np.sum(H[i][:]))

area_frequent = np.where(H_sum == np.max(H_sum))[0][0]

# definitions for the axes
left, width = 1, 1
bottom, height = 1, 1
bottom_h = left_h = left + width + 0.025

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# start with a rectangular Figure
plt.figure(1, figsize=(8, 8))

axScatter = plt.axes(rect_scatter)
axHistx = plt.axes(rect_histx)
axHisty = plt.axes(rect_histy)

axScatter.pcolormesh(xedges,yedges,Hmasked,norm=LogNorm())
axScatter.set_xlabel('CH0 area [PE]')
axScatter.set_ylabel('Area ratio CH0/CH1')
axScatter.set_xlim(min(ch0_area),max(ch0_area))
axScatter.hlines(y=area_average,xmin=np.min(ch0_area),xmax=np.max(ch0_area),
           linestyle='--',color='orange',
           label='average area ratio = '+str(round(area_average,1)))
axScatter.hlines(y=area_frequent,xmin=np.min(ch0_area),xmax=np.max(ch0_area),
           linestyle='--',color='red',
           label='most frequent area ratio = '+str(area_frequent))
axScatter.legend()

axHistx.hist(filtered_ch0_area,bins=200,histtype='step',linewidth=3,color='darkblue')
axHistx.set_xlim(min(ch0_area),max(ch0_area))
axHistx.set_xticks([])
axHistx.set_ylabel('Counts')

axHisty.hist(area_ratio,bins=200,range=(0,200),orientation='horizontal',histtype='step',linewidth=3,color='darkblue')
axHisty.set_ylim(0,200)
axHisty.set_yticks([])
axHisty.set_xlabel('Counts')

plt.suptitle('Pulse area ratio against area without polariser, '+str(V)+' V, '+str(T)+' ns',x=1.55,y=2.275)

plt.savefig('./img/Pulse area ratio with and without polariser, '+str(V)+' V, '+str(T)+'ns.png',dpi=300)
plt.show()

### CH1 Histogram - P

In [ ]:
# Use numpy.hist() function to calculate histogram data
ch1_hist,ch1_bin_edges = np.histogram(ch1_area,bins=200,range=(0,max(ch1_area)))
ch1_g = ch1_hist
ch1_x = (ch1_bin_edges[:-1] + ch1_bin_edges[1:])/2

# Define non-normalised Gaussian function
def poisson_fit(x,A,mu,x_0):
  g = A * np.exp(-mu)*np.power(mu, (x-x_0))/factorial(x-x_0)
  return g

# Curve fit the histgram data using curve_fit()
ch1_popt,ch1_pcov = curve_fit(poisson_fit,ch1_x,ch1_g,p0=(1E5,20,0))
ch1_perr = np.sqrt(np.diag(ch1_pcov))

# Report the parameters
print('A =', round(ch1_popt[0],3), '+/-', round(ch1_perr[0],3))
print('mu =', round(ch1_popt[1],3), '+/-', round(ch1_perr[1],3))
print('x_0 =', round(ch1_popt[2],3), '+/-', round(ch1_perr[2],3))

# prepare for the fitted curve
ch1_u = np.arange(0,max(ch1_area),0.1)
ch1_v = poisson_fit(ch1_u,ch1_popt[0],ch1_popt[1],ch1_popt[2])

plt.figure(figsize=(16,9))
plt.title('Histogram of CH1 area, '+str(V)+' V, '+str(T)+'ns, '+str(PDiv)+'dB')
plt.xlabel('CH1 area [PE]')
plt.ylabel('Counts')
plt.hist(ch1_area,bins=200,color='silver',label='CH0 area')
plt.plot(ch1_u,ch1_v,color='darkblue',linewidth=3,
         label='Fitted curve, $\mu$ = '+str(round(ch1_popt[1],3))+', $x_0$ = '+str(round(ch1_popt[2],3)))
plt.legend()
plt.savefig('./img/Histogram of CH1 area, '+str(V)+' V, '+str(T)+'ns.png',dpi=300)
plt.show()

### CH1 histogram - G

In [ ]:
# Use numpy.hist() function to calculate histogram data
ch1_hist,ch1_bin_edges = np.histogram(ch1_area,bins=200)
ch1_g = ch1_hist
ch1_x = (ch1_bin_edges[:-1] + ch1_bin_edges[1:])/2

# Define non-normalised Gaussian function
def gaussian(x,A,mu,sigma):
  g = A * np.exp(-(x-mu)**2 / (2*sigma**2))
  return g

# Curve fit the histgram data using curve_fit()
ch1_popt,ch1_pcov = curve_fit(gaussian,ch1_x,ch1_g,p0=(1000,(min(ch1_area)+max(ch1_area))/2,50))
ch1_perr = np.sqrt(np.diag(ch1_pcov))

# Report the parameters
print('A =', round(ch1_popt[0],3), '+/-', round(ch1_perr[0],3))
print('mu =', round(ch1_popt[1],3), '+/-', round(ch1_perr[1],3))
print('sigma =', round(ch1_popt[2],3), '+/-', round(ch1_perr[2],3))

# prepare for the fitted curve
ch1_u = np.arange(min(ch1_area),max(ch1_area),0.1)
ch1_v = round(ch1_popt[0],3) * np.exp(-(ch1_u-round(ch1_popt[1],3))**2 / (2*round(ch1_popt[2],3)**2))

plt.figure(figsize=(16,9))
plt.title('Histogram of CH1 area, '+str(V)+' V, '+str(T)+'ns, '+str(PDiv)+'dB')
plt.xlabel('CH1 area [PE]')
plt.ylabel('Counts')
plt.hist(ch1_area,bins=200,color='silver',label='CH1 area')
plt.plot(ch1_u,ch1_v,color='darkblue',linewidth=3,
         label='Fitted curve, $\mu$ = '+str(round(ch1_popt[1],3))+', $\sigma$ = '+str(np.abs(round(ch1_popt[2],3))))
plt.legend()
plt.savefig('./img/Histogram of CH1 area, '+str(V)+' V, '+str(T)+'ns.png',dpi=300)
plt.show()

### CH2 Histogram - G

In [ ]:
# Use numpy.hist() function to calculate histogram data
ch2_hist,ch2_bin_edges = np.histogram(ch2_area,bins=200)
ch2_g = ch2_hist
ch2_x = (ch2_bin_edges[:-1] + ch2_bin_edges[1:])/2

# Define non-normalised Gaussian function
def gaussian(x,A,mu,sigma):
  g = A * np.exp(-(x-mu)**2 / (2*sigma**2))
  return g

# Curve fit the histgram data using curve_fit()
ch2_popt,ch2_pcov = curve_fit(gaussian,ch2_x,ch2_g,p0=(1000,(min(ch2_area)+max(ch2_area))/2,50))
ch2_perr = np.sqrt(np.diag(ch2_pcov))

# Report the parameters
print('A =', round(ch2_popt[0],3), '+/-', round(ch2_perr[0],3))
print('mu =', round(ch2_popt[1],3), '+/-', round(ch2_perr[1],3))
print('sigma =', round(ch2_popt[2],3), '+/-', round(ch2_perr[2],3))

# prepare for the fitted curve
ch2_u = np.arange(min(ch2_area),max(ch2_area),0.1)
ch2_v = round(ch2_popt[0],3) * np.exp(-(ch2_u-round(ch2_popt[1],3))**2 / (2*round(ch2_popt[2],3)**2))

plt.figure(figsize=(16,9))
plt.title('Histogram of CH2 area, '+str(V)+' V, '+str(T)+'ns, '+str(PDiv)+'dB')
plt.xlabel('CH2 area [PE]')
plt.ylabel('Counts')
plt.hist(ch2_area,bins=200,color='silver',label='CH2 area')
plt.plot(ch2_u,ch2_v,color='darkblue',linewidth=3,
         label='Fitted curve, $\mu$ = '+str(round(ch2_popt[1],3))+', $\sigma$ = '+str(np.abs(round(ch2_popt[2],3))))
plt.legend()
#plt.savefig('./img/Histogram of CH2 area, '+str(V)+' V, '+str(T)+'ns.png',dpi=300)
plt.show()

### CH0 Histogram - G

In [ ]:
# Use numpy.hist() function to calculate histogram data
ch0_hist,ch0_bin_edges = np.histogram(ch0_area,bins=200)
ch0_g = ch0_hist
ch0_x = (ch0_bin_edges[:-1] + ch0_bin_edges[1:])/2

# Define non-normalised Gaussian function
def gaussian(x,A,mu,sigma):
  g = A * np.exp(-(x-mu)**2 / (2*sigma**2))
  return g

# Curve fit the histgram data using curve_fit()
ch0_popt,ch0_pcov = curve_fit(gaussian,ch0_x,ch0_g,p0=(1000,(min(ch0_area)+max(ch0_area))/2,50))
ch0_perr = np.sqrt(np.diag(ch0_pcov))

# Report the parameters
print('A =', round(ch0_popt[0],3), '+/-', round(ch0_perr[0],3))
print('mu =', round(ch0_popt[1],3), '+/-', round(ch0_perr[1],3))
print('sigma =', round(ch0_popt[2],3), '+/-', round(ch0_perr[2],3))

# prepare for the fitted curve
ch0_u = np.arange(min(ch0_area),max(ch0_area),0.1)
ch0_v = round(ch0_popt[0],3) * np.exp(-(ch0_u-round(ch0_popt[1],3))**2 / (2*round(ch0_popt[2],3)**2))

plt.figure(figsize=(16,9))
plt.title('Histogram of CH0 area, '+str(V)+' V, '+str(T)+'ns, '+str(PDiv)+'dB')
plt.xlabel('CH0 area [PE]')
plt.ylabel('Counts')
plt.hist(ch0_area,bins=200,color='silver',label='CH0 area')
plt.plot(ch0_u,ch0_v,color='darkblue',linewidth=3,
         label='Fitted curve, $\mu$ = '+str(round(ch0_popt[1],3))+', $\sigma$ = '+str(np.abs(round(ch0_popt[2],3))))
plt.legend()
#plt.savefig('./img/Histogram of CH0 area, '+str(V)+' V, '+str(T)+'ns.png',dpi=300)
plt.show()

### Ratio Histogram

In [ ]:
# Use numpy.hist() function to calculate histogram data
ratio_hist, ratio_bin_edges = np.histogram(area_ratio,bins=200,range=(0,200))
ratio_g = ratio_hist
ratio_x = (ratio_bin_edges[:-1] + ratio_bin_edges[1:])/2

# Define the fitting function
def landau_fit(x, A, location, scale):
    return A * landau.pdf(x=x, x_mpv=location, xi=scale)

# Fit the data
ratio_popt, ratio_pcov = curve_fit(landau_fit, ratio_x, ratio_g, p0=[1E5, 30, 5])
ratio_perr = np.sqrt(np.diag(ratio_pcov))

# Report parameters
print('A =', round(ratio_popt[0],3), '+/-', round(ratio_perr[0],3))
print('location =', round(ratio_popt[1],3), '+/-', round(ratio_perr[1],3))
print('scale =', round(ratio_popt[2],3), '+/-', round(ratio_perr[2],3))

# Generate the fitted Landau distribution
ratio_u = np.linspace(0,200, 400)
ratio_v = landau_fit(ratio_u, ratio_popt[0], ratio_popt[1], ratio_popt[2])

plt.figure(figsize=(16,9))
plt.title('Histogram of the area ratio, '+str(V)+' V, '+str(T)+'ns, '+str(PDiv)+'dB')
plt.xlabel('Area ratio')
plt.ylabel('Counts')
plt.xlim(0,200)
plt.hist(area_ratio,bins=200,range=(0,200),color='silver',label='Histogram')
plt.plot(ratio_u, ratio_v, 'r-',color='darkblue',linewidth=3,label='Fitted function, location = '+str(round(ratio_popt[1],1)))
plt.legend()
plt.savefig('./img/Histogram of the area ratio, '+str(V)+' V, '+str(T)+'ns.png',dpi=300)
plt.show()

### Generalise 

In [ ]:
10**(6/20)

In [ ]:
2.76 (2.71656554) # experimental results

In [ ]:
general = np.genfromtxt('./linear.txt',skip_header=1)

div_ratio = []
for i in range (0,len(general[:,0])):
    if general[i,6]==0:
        div_ratio.append(1)
    elif general[i,6]==9:
        div_ratio.append(2.71656554) # 10**(general[i,6]/20)

"""
def exp_fit(x,A_0,A,x_0,alpha):
    return A_0 - (A * np.exp(-(x-x_0)/alpha))

general_popt, general_pcov = curve_fit(exp_fit,general[6:,0],general[6:,2],p0=(40,40,10,150))

print('A_0 =',general_popt[0])
print('A =',general_popt[1])
print('alpha =',general_popt[3])

general_fit_x = np.linspace(0,1000,1000)
general_fit_y = exp_fit(general_fit_x,general_popt[0],general_popt[1],general_popt[2],general_popt[3])

plt.figure(figsize=(16,9))
plt.title('Linearity of RELICS PMT base')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('CH0 area [PE]')
plt.ylabel('CH1 area [PE]')
plt.xlim(10,10000)

#plt.scatter(general[:,4],general[:,2])

plt.errorbar(general[:,2]*div_ratio[:],general[:,4],xerr=general[:,3]*div_ratio[:],
             yerr=general[:,5],fmt='o',ecolor='darkcyan')

#plt.plot(general_fit_x,general_fit_y,color='orange',label='Fitted function, limit = '+str(round(general_popt[0],2)))

#plt.legend()
plt.savefig('./img/area_linear_generalised.png',dpi=300)
plt.show()
"""

In [ ]:
general_ratio = general[:,2]*div_ratio[:] / general[:,4]
general_ratio_error = general_ratio * np.sqrt((general[:,3]/general[:,2])**2+(general[:,5]/general[:,4])**2)

plt.figure(figsize=(16,9))
plt.title('Area ratio vs CH0 area')
plt.xscale('log')
plt.xlabel('CH0 area [PE]')
plt.ylabel('Area ratio')
plt.xlim(330,10000)
plt.ylim(0,100)

plt.errorbar(general[:20,2]*div_ratio[:20],general_ratio[:20],xerr=general[:20,3]*div_ratio[:20],yerr=general_ratio_error[:20],
             fmt='o',color = 'royalblue',label='0 dB')
plt.errorbar(general[21:26,2]*div_ratio[21:26],general_ratio[21:26],xerr=general[21:26,3]*div_ratio[21:26],yerr=general_ratio_error[21:26],
             fmt='o',color = 'green',label='20240910, 9 dB')
plt.errorbar(general[26:,2]*div_ratio[26:],general_ratio[26:],xerr=general[26:,3]*div_ratio[26:],yerr=general_ratio_error[26:],
             fmt='o',color = 'darkorange',label='20240920, 9 dB')
plt.hlines(y = 51.27,color = 'red',linestyles='dashed',xmin = 0,xmax = 10000,label='Linear ratio')
plt.vlines(x = 1000,ymin = -50,ymax = 170, color = 'cyan',
           linestyles='dashed',label='1000 PE limit')

plt.legend()
plt.savefig('./img/area_ratio_generalised.png',dpi = 300)
plt.show()

In [ ]:
dynode = np.genfromtxt('./linear_dynode.txt',skip_header=1)

din_ratio = []
for i in range (0,len(dynode[:,0])):
    if dynode[i,6]==0:
        din_ratio.append(1)
    elif dynode[i,6]==9:
        din_ratio.append(2.71656554) # 10**(general[i,6]/20)

dynode_ratio = dynode[:,2]*din_ratio[:] / dynode[:,4]
dynode_ratio_error = dynode_ratio * np.sqrt((dynode[:,3]/dynode[:,2])**2+(dynode[:,5]/dynode[:,4])**2)

plt.figure(figsize=(16,9))
plt.title('Dynode ratio vs CH0 area')
plt.xscale('log')
plt.xlabel('CH0 area [PE]')
plt.ylabel('Dynode ratio')
plt.xlim(330,10000)
plt.ylim(0,200)

plt.errorbar(dynode[9:18,2]*din_ratio[9:18],dynode_ratio[9:18],xerr=dynode[9:18,3]*din_ratio[9:18],yerr=dynode_ratio_error[9:18],
             fmt='o',color = 'royalblue',label='0 dB')
plt.errorbar(dynode[21:25,2]*din_ratio[21:25],dynode_ratio[21:25],xerr=dynode[21:25,3]*din_ratio[21:25],yerr=dynode_ratio_error[21:25],
             fmt='o',color = 'green',label='20240910, 9 dB')
plt.errorbar(dynode[25:,2]*din_ratio[25:],dynode_ratio[25:],xerr=dynode[25:,3]*din_ratio[25:],yerr=dynode_ratio_error[25:],
             fmt='o',color = 'darkorange',label='20240920, 9 dB')
plt.hlines(y = 151.91,color = 'red',linestyles='dashed',xmin = 0,xmax = 10000,label='Dynode ratio')
plt.vlines(x = 1000,ymin = -50,ymax = 200, color = 'cyan',
           linestyles='dashed',label='1000 PE limit')

plt.legend()
plt.savefig('./img/dynode_ratio_generalised.png',dpi = 300)
plt.show()